# National generation and fuel consumption
The data in this notebook is generation and consumption by fuel type for the entire US. These values are larger than what would be calculated by summing facility-level data. Note that the fuel types are somewhat aggregated (coal rather than BIT, SUB, LIG, etc). So when we multiply the fuel consumption by an emissions factor there will be some level of error.

The code assumes that you have already downloaded an `ELEC.txt` file from [EIA's bulk download website](https://www.eia.gov/opendata/bulkfiles.php).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import io, time, json
import pandas as pd
import os
from os.path import join
import numpy as np
import math

In [77]:
idx = pd.IndexSlice

## Read ELECT.txt file

In [2]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage', 'Raw EIA bulk',
            '2017-08-31 ELEC_manual.txt')
with open(path, 'rb') as f:
    raw_txt = f.readlines()

## Filter lines to only include total generation and fuel use
Only want monthly US data for all sectors
- US-99.M
- ELEC.GEN, ELEC.CONS_TOT_BTU, ELEC.CONS_EG_BTU
- not ALL

Fuel codes:
- WWW, wood and wood derived fuels
- WND, wind
- STH, solar thermal
- WAS, other biomass
- TSN, all solar
- SUN, utility-scale solar
- NUC, nuclear
- NG, natural gas
- PEL, petroleum liquids
- SPV, utility-scale solar photovoltaic
- PC, petroluem coke
- OTH, other
- COW, coal,
- DPV, distributed photovoltaic
- OOG, other gases
- HPS, hydro pumped storage
- HYC, conventional hydroelectric
- GEO, geothermal
- AOR, other renewables (total)

In [3]:
def line_to_df(line):
    """
    Takes in a line (dictionary), returns a dataframe
    """
    for key in ['latlon', 'source', 'copyright', 'description', 
                'geoset_id', 'iso3166', 'name', 'state']:
        line.pop(key, None)

    # Split the series_id up to extract information
    # Example: ELEC.PLANT.GEN.388-WAT-ALL.M
    series_id = line['series_id']
    series_id_list = series_id.split('.')
    # Use the second to last item in list rather than third
    plant_fuel_mover = series_id_list[-2].split('-')
    line['type'] = plant_fuel_mover[0]
#     line['state'] = plant_fuel_mover[1]
    line['sector'] = plant_fuel_mover[2]
    temp_df = pd.DataFrame(line)

    try:
        temp_df['year'] = temp_df.apply(lambda x: x['data'][0][:4], axis=1).astype(int)
        temp_df['month'] = temp_df.apply(lambda x: x['data'][0][-2:], axis=1).astype(int)
        temp_df['value'] = temp_df.apply(lambda x: x['data'][1], axis=1)
        temp_df.drop('data', axis=1, inplace=True)
        return temp_df
    except:
        exception_list.append(line)
        pass

In [4]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE",
          "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS",
          "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS",
          "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
          "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [17]:
state_geos = ['USA-{}'.format(state) for state in states]

In [8]:
type(json.loads(raw_txt[0]))

dict

In [10]:
json.loads(raw_txt[0])['geography']

u'USA-IL'

In [12]:
exception_list = []
gen_rows = [row for row in raw_txt if 'ELEC.GEN' in row 
            and 'series_id' in row 
            and '-99.M' in row 
            and 'ALL' not in row]

total_fuel_rows = [row for row in raw_txt if 'ELEC.CONS_TOT_BTU' in row 
                   and 'series_id' in row 
                   and '-99.M' in row 
                   and 'ALL' not in row
                   and 'US-99.m' not in row]

eg_fuel_rows = [row for row in raw_txt if 'ELEC.CONS_EG_BTU' in row 
                and 'series_id' in row 
                and '-99.M' in row 
                and 'ALL' not in row
                and 'US-99.m' not in row]

## All generation by fuel

In [13]:
gen_dicts = [json.loads(row) for row in gen_rows]

In [143]:
gen_df = pd.concat([line_to_df(x) for x in gen_dicts
                    if x['geography'] in state_geos])

In [144]:
#drop
gen_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,value
0,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,6,1066.98487
1,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,5,1173.59609
2,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,4,1250.62552
3,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,3,1345.11365
4,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,2,1222.31074


In [145]:
gen_df['geography'].unique()

array([u'USA-MN', u'USA-KY', u'USA-LA', u'USA-CA', u'USA-ME', u'USA-DE',
       u'USA-ID', u'USA-ND', u'USA-AR', u'USA-MI', u'USA-MA', u'USA-KS',
       u'USA-HI', u'USA-NH', u'USA-FL', u'USA-NC', u'USA-MD', u'USA-AK',
       u'USA-CO', u'USA-AZ', u'USA-AL', u'USA-NJ', u'USA-IL', u'USA-CT',
       u'USA-MT', u'USA-IA', u'USA-GA', u'USA-OK', u'USA-NY', u'USA-IN',
       u'USA-NV', u'USA-NM', u'USA-WA', u'USA-OH', u'USA-TX', u'USA-RI',
       u'USA-OR', u'USA-WY', u'USA-SD', u'USA-PA', u'USA-VA', u'USA-VT',
       u'USA-UT', u'USA-SC', u'USA-WI', u'USA-WV', u'USA-NE', u'USA-TN',
       u'USA-MO', u'USA-MS'], dtype=object)

Multiply generation values by 1000 and change the units to MWh

In [146]:
gen_df.loc[:,'value'] *= 1000
gen_df.loc[:,'units'] = 'megawatthours'

In [147]:
gen_df.rename_axis({'value':'generation (MWh)'}, axis=1, inplace=True)

In [148]:
gen_df.loc[gen_df.isnull().any(axis=1)]

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh)
4,201706,M,USA-AK,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-AK-99.M,200101,AOR,megawatthours,2017,2,NaN
5,201706,M,USA-AK,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-AK-99.M,200101,AOR,megawatthours,2017,1,NaN
55,201706,M,USA-AK,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-AK-99.M,200101,AOR,megawatthours,2012,11,NaN
65,201706,M,USA-AK,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-AK-99.M,200101,AOR,megawatthours,2012,1,NaN
0,201706,M,USA-ID,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-ID-99.M,200101,COW,megawatthours,2017,6,NaN
1,201706,M,USA-ID,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-ID-99.M,200101,COW,megawatthours,2017,5,NaN
2,201706,M,USA-ID,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-ID-99.M,200101,COW,megawatthours,2017,4,NaN
3,201706,M,USA-ID,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-ID-99.M,200101,COW,megawatthours,2017,3,NaN
4,201706,M,USA-ID,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-ID-99.M,200101,COW,megawatthours,2017,2,NaN
5,201706,M,USA-ID,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-ID-99.M,200101,COW,megawatthours,2017,1,NaN


In [149]:
gen_df.dropna(inplace=True)

In [150]:
gen_df.set_index(['type', 'year', 'month', 'geography'], inplace=True)

In [151]:
#drop
gen_df.head()

end  f               last_updated sector  \
type year month geography                                                
AOR  2017 6     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   

                                      series_id   start          units  \
type year month geography                                                
AOR  2017 6     USA-MN     ELEC.GEN.AOR-MN-99.M  200101  megawatthours   
          5     USA-MN     ELEC.GEN.AOR-MN-99.M  200101  megawatthours   
          4     USA-MN     ELEC.GEN.AOR-MN-99.M  200101  megawatthours   
          3     USA-MN     ELEC.GEN.AOR-MN-99.M  200101  megawatthours   
          2     USA-MN     ELEC.GEN.AOR-MN-99.M  200101  megawatthours   

                           generation (MWh)  
type year month geography                    
AOR  2017 6     USA-MN           1066984.87  
          5     USA-MN           1173596.09  
          4     USA-MN           1250625.52  
          3     USA-MN           1345113.65  
          2     USA-MN           1222310.74

In [152]:
#drop
gen_df.loc['OOG'].head()

end  f               last_updated sector  \
year month geography                                                
2017 6     USA-NV     201706  M  2017-08-24T11:46:12-04:00     99   
     5     USA-NV     201706  M  2017-08-24T11:46:12-04:00     99   
     4     USA-NV     201706  M  2017-08-24T11:46:12-04:00     99   
     3     USA-NV     201706  M  2017-08-24T11:46:12-04:00     99   
     2     USA-NV     201706  M  2017-08-24T11:46:12-04:00     99   

                                 series_id   start          units  \
year month geography                                                
2017 6     USA-NV     ELEC.GEN.OOG-NV-99.M  200210  megawatthours   
     5     USA-NV     ELEC.GEN.OOG-NV-99.M  200210  megawatthours   
     4     USA-NV     ELEC.GEN.OOG-NV-99.M  200210  megawatthours   
     3     USA-NV     ELEC.GEN.OOG-NV-99.M  200210  megawatthours   
     2     USA-NV     ELEC.GEN.OOG-NV-99.M  200210  megawatthours   

                      generation (MWh)  
year month geography                    
2017 6     USA-NV                  0.0  
     5     USA-NV                  0.0  
     4     USA-NV                  0.0  
     3     USA-NV                  0.0  
     2     USA-NV                  0.0

## Total fuel consumption

In [39]:
total_fuel_dict = [json.loads(row) for row in total_fuel_rows]

In [153]:
total_fuel_df = pd.concat([line_to_df(x) for x in total_fuel_dict
                           if x['geography'] in state_geos])

In [154]:
#drop
total_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,value
0,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,6,0.0
1,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,5,0.0
2,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,4,0.0
3,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,3,0.0
4,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,2,0.0


Multiply generation values by 1,000,000 and change the units to MMBtu

In [155]:
total_fuel_df.loc[:,'value'] *= 1E6
total_fuel_df.loc[:,'units'] = 'mmbtu'

In [156]:
total_fuel_df.rename_axis({'value':'total fuel (mmbtu)'}, axis=1, inplace=True)

In [157]:
total_fuel_df.set_index(['type', 'year', 'month', 'geography'], inplace=True)

In [158]:
#drop
total_fuel_df.head()

end  f               last_updated sector  \
type year month geography                                                
PC   2017 6     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   

                                              series_id   start  units  \
type year month geography                                                
PC   2017 6     USA-CA     ELEC.CONS_TOT_BTU.PC-CA-99.M  200101  mmbtu   
          5     USA-CA     ELEC.CONS_TOT_BTU.PC-CA-99.M  200101  mmbtu   
          4     USA-CA     ELEC.CONS_TOT_BTU.PC-CA-99.M  200101  mmbtu   
          3     USA-CA     ELEC.CONS_TOT_BTU.PC-CA-99.M  200101  mmbtu   
          2     USA-CA     ELEC.CONS_TOT_BTU.PC-CA-99.M  200101  mmbtu   

                           total fuel (mmbtu)  
type year month geography                      
PC   2017 6     USA-CA                    0.0  
          5     USA-CA                    0.0  
          4     USA-CA                    0.0  
          3     USA-CA                    0.0  
          2     USA-CA                    0.0

In [159]:
#drop
total_fuel_df.loc[~(total_fuel_df['total fuel (mmbtu)'] >= 0) &
                  ~(total_fuel_df['total fuel (mmbtu)'].isnull())]

,,,,end,f,last_updated,sector,series_id,start,units,total fuel (mmbtu)
type,year,month,geography,,,,,,,,


Drop nans

In [160]:
total_fuel_df.loc[total_fuel_df.isnull().any(axis=1)]

end  f               last_updated sector  \
type year month geography                                                
PC   2012 12    USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          11    USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          10    USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          9     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          8     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          7     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          6     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
     2016 11    USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          10    USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          9     USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          8     USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          6     USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
          1     USA-IA     201706  M  2017-08-24T11:46:12-04:00     99   
NG   2016 6     USA-WY     201706  M  2017-08-24T11:46:12-04:00     99   
PC   2017 6     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
          1     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
     2014 5     USA-GA     201706  M  2017-08-24T11:46:12-04:00     99   
COW  2017 6     USA-ID     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-ID     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-ID     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-ID     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-ID     201706  M  2017-08-24T11:46:12-04:00     99   
...                           ... ..                        ...    ...   
NG   2016 8     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          7     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          6     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
          1     USA-VT     201706  M  2017-08-24T11:46:12-04:00     99   
PEL  2017 3     USA-VA     201706  M  2017-08-24T11:46:12-04:00     99   
          6     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          1     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
     2016 12    USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          11    USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          10    USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          9     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          8     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          7     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          6     USA-CO     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-CO     201706 

In [161]:
total_fuel_df = total_fuel_df.dropna()

## Electric generation fuel consumption

In [52]:
eg_fuel_dict = [json.loads(row) for row in eg_fuel_rows]

In [162]:
eg_fuel_df = pd.concat([line_to_df(x) for x in eg_fuel_dict
                        if x['geography'] in state_geos])

In [163]:
#drop
eg_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,value
0,201706,M,USA-VA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-VA-99.M,200101,NG,million MMBtu,2017,6,35.91854
1,201706,M,USA-VA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-VA-99.M,200101,NG,million MMBtu,2017,5,24.96651
2,201706,M,USA-VA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-VA-99.M,200101,NG,million MMBtu,2017,4,21.21269
3,201706,M,USA-VA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-VA-99.M,200101,NG,million MMBtu,2017,3,28.41358
4,201706,M,USA-VA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-VA-99.M,200101,NG,million MMBtu,2017,2,24.13910


Multiply generation values by 1,000,000 and change the units to MMBtu

In [164]:
eg_fuel_df.loc[:,'value'] *= 1E6
eg_fuel_df.loc[:,'units'] = 'mmbtu'

In [165]:
eg_fuel_df.rename_axis({'value':'elec fuel (mmbtu)'}, axis=1, inplace=True)

In [166]:
eg_fuel_df.set_index(['type', 'year', 'month', 'geography'], inplace=True)

In [167]:
#drop
eg_fuel_df.head()

end  f               last_updated sector  \
type year month geography                                                
NG   2017 6     USA-VA     201706  M  2017-08-24T11:46:12-04:00     99   
          5     USA-VA     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-VA     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-VA     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-VA     201706  M  2017-08-24T11:46:12-04:00     99   

                                             series_id   start  units  \
type year month geography                                               
NG   2017 6     USA-VA     ELEC.CONS_EG_BTU.NG-VA-99.M  200101  mmbtu   
          5     USA-VA     ELEC.CONS_EG_BTU.NG-VA-99.M  200101  mmbtu   
          4     USA-VA     ELEC.CONS_EG_BTU.NG-VA-99.M  200101  mmbtu   
          3     USA-VA     ELEC.CONS_EG_BTU.NG-VA-99.M  200101  mmbtu   
          2     USA-VA     ELEC.CONS_EG_BTU.NG-VA-99.M  200101  mmbtu   

                           elec fuel (mmbtu)  
type year month geography                     
NG   2017 6     USA-VA            35918540.0  
          5     USA-VA            24966510.0  
          4     USA-VA            21212690.0  
          3     USA-VA            28413580.0  
          2     USA-VA            24139100.0

I verified on [EIA's website](https://www.eia.gov/opendata/qb.php?category=400&sdid=ELEC.CONS_EG_BTU.PEL-MN-99.M) that the values below are correct.

In [168]:
#drop
eg_fuel_df.loc[~(eg_fuel_df['elec fuel (mmbtu)'] >= 0) &
                  ~(eg_fuel_df['elec fuel (mmbtu)'].isnull())]

end  f               last_updated sector  \
type year month geography                                                
PEL  2002 12    USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          11    USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          10    USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          8     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          7     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          1     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   

                                              series_id   start  units  \
type year month geography                                                
PEL  2002 12    USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          11    USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          10    USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          8     USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          7     USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          4     USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          3     USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          2     USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   
          1     USA-MN     ELEC.CONS_EG_BTU.PEL-MN-99.M  200101  mmbtu   

                           elec fuel (mmbtu)  
type year month geography                     
PEL  2002 12    USA-MN              -43000.0  
          11    USA-MN              -32000.0  
          10    USA-MN              -15000.0  
          8     USA-MN              -16000.0  
          7     USA-MN               -1000.0  
          4     USA-MN               -6000.0  
          3     USA-MN              -10000.0  
          2     USA-MN              -30000.0  
          1     USA-MN              -34000.0

In [169]:
eg_fuel_df.dropna(inplace=True)

## Combine three datasets
Need to estimate fuel use for OOG, because EIA doesn't include any (this is only ~2% of OOG fuel for electricity in 2015).

In [170]:
fuel_df = pd.concat([total_fuel_df, eg_fuel_df['elec fuel (mmbtu)']], axis=1)

Not sure how this happens in EIA's data, but we do see the negative fuel consumption for electricity generation.

In [171]:
#drop
fuel_df.loc[~(fuel_df['elec fuel (mmbtu)']>=0)]

end  f               last_updated sector  \
type year month geography                                                
PEL  2002 1     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          2     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          3     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          4     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          7     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          8     USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          10    USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          11    USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   
          12    USA-MN     201706  M  2017-08-24T11:46:12-04:00     99   

                                               series_id   start  units  \
type year month geography                                                 
PEL  2002 1     USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          2     USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          3     USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          4     USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          7     USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          8     USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          10    USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          11    USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   
          12    USA-MN     ELEC.CONS_TOT_BTU.PEL-MN-99.M  200101  mmbtu   

                           total fuel (mmbtu)  elec fuel (mmbtu)  
type year month geography                                         
PEL  2002 1     USA-MN                51000.0           -34000.0  
          2     USA-MN                62000.0           -30000.0  
          3     USA-MN                99000.0           -10000.0  
          4     USA-MN                84000.0            -6000.0  
          7     USA-MN                93000.0            -1000.0  
          8     USA-MN                66000.0           -16000.0  
          10    USA-MN                64000.0           -15000.0  
          11    USA-MN                49000.0           -32000.0  
          12    USA-MN                50000.0           -43000.0

In [172]:
#drop
fuel_df.loc[~(fuel_df['total fuel (mmbtu)']>=0)]

,,,,end,f,last_updated,sector,series_id,start,units,total fuel (mmbtu),elec fuel (mmbtu)
type,year,month,geography,,,,,,,,,


### Add CO<sub>2</sub> emissions

The difficulty here is that EIA combines all types of coal fuel consumption together in the bulk download and API. Fortunately the emission factors for different coal types aren't too far off on an energy basis (BIT is 93.3 kg/mmbtu, SUB is 97.2 kg/mmbtu). I'm going to average the BIT and SUB factors rather than trying to do something more complicated. In 2015 BIT represented 45% of coal energy for electricity and SUB represented 48%.

Same issue with petroleum liquids. Using the average of DFO and RFO, which were the two largest share of petroleum liquids.

In [173]:
path = join(cwd, '..', 'Data storage', 'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [174]:
#drop
ef.index

Index([u'BIT', u'DFO', u'GEO', u'JF', u'KER', u'LIG', u'MSW', u'NG', u'PC',
       u'PG', u'RC', u'RFO', u'SGC', u'SGP', u'SUB', u'TDF', u'WC', u'WO',
       u'BFG', u'MSN', u'SC', u'OG', u'AB', u'BLQ', u'LFG', u'MSB', u'NUC',
       u'OBG', u'OBL', u'OBS', u'OTH', u'PUR', u'SLW', u'SUN', u'WAT', u'WDL',
       u'WDS', u'WH', u'WND'],
      dtype='object', name=u'EIA Fuel Code')

### Match general types with specific fuel codes

Fuel codes:
- WWW, wood and wood derived fuels
- WND, wind
- STH, solar thermal
- WAS, other biomass
- TSN, all solar
- SUN, utility-scale solar
- NUC, nuclear
- NG, natural gas
- PEL, petroleum liquids
- SPV, utility-scale solar photovoltaic
- PC, petroluem coke
- OTH, other
- COW, coal,
- DPV, distributed photovoltaic
- OOG, other gases
- HPS, hydro pumped storage
- HYC, conventional hydroelectric
- GEO, geothermal
- AOR, other renewables (total)

In [175]:
#drop
ef.loc['NG', 'Fossil Factor']

53.07

In [176]:
fuel_factors = pd.Series({'NG' : ef.loc['NG', 'Fossil Factor'],
                   'PEL': ef.loc[['DFO', 'RFO'], 'Fossil Factor'].mean(),
                   'PC' : ef.loc['PC', 'Fossil Factor'], 
                   'COW' : ef.loc[['BIT', 'SUB'], 'Fossil Factor'].mean(),
                   'OOG' : ef.loc['OG', 'Fossil Factor']}, name='type')

In [177]:
#drop
fuel_factors

COW     95.250
NG      53.070
OOG     59.000
PC     102.100
PEL     75.975
Name: type, dtype: float64

In [178]:
fuel_df.index.get_level_values('type').unique()

Index([u'COW', u'NG', u'PC', u'PEL'], dtype='object', name=u'type')

In [179]:
fuel_df['all fuel CO2 (kg)'] = (fuel_df['total fuel (mmbtu)']
                                .multiply(fuel_factors, level='type',
                                          fill_value=0))
fuel_df['elec fuel CO2 (kg)'] = (fuel_df['elec fuel (mmbtu)']
                                .multiply(fuel_factors, level='type',
                                          fill_value=0))

In [180]:
fuel_df.head()

end  f               last_updated sector  \
type year month geography                                                
COW  2001 1     USA-AK     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-AL     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-AR     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-AZ     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   

                                               series_id   start  units  \
type year month geography                                                 
COW  2001 1     USA-AK     ELEC.CONS_TOT_BTU.COW-AK-99.M  200101  mmbtu   
                USA-AL     ELEC.CONS_TOT_BTU.COW-AL-99.M  200101  mmbtu   
                USA-AR     ELEC.CONS_TOT_BTU.COW-AR-99.M  200101  mmbtu   
                USA-AZ     ELEC.CONS_TOT_BTU.COW-AZ-99.M  200101  mmbtu   
                USA-CA     ELEC.CONS_TOT_BTU.COW-CA-99.M  200101  mmbtu   

                           total fuel (mmbtu)  elec fuel (mmbtu)  \
type year month geography                                          
COW  2001 1     USA-AK              1120000.0           872000.0   
                USA-AL             67999000.0         66582000.0   
                USA-AR             23099000.0         22700000.0   
                USA-AZ             35873000.0         35483000.0   
                USA-CA              3652000.0          2008000.0   

                           all fuel CO2 (kg)  elec fuel CO2 (kg)  
type year month geography                                         
COW  2001 1     USA-AK          1.066800e+08        8.305800e+07  
                USA-AL          6.476905e+09        6.341935e+09  
                USA-AR          2.200180e+09        2.162175e+09  
                USA-AZ          3.416903e+09        3.379756e+09  
                USA-CA          3.478530e+08        1.912620e+08

In [181]:
fuel_cols = ['total fuel (mmbtu)', 'elec fuel (mmbtu)',
              'all fuel CO2 (kg)', 'elec fuel CO2 (kg)']
gen_fuel_df = pd.concat([gen_df, fuel_df[fuel_cols]], axis=1)

In [182]:
gen_fuel_df.head()

end  f               last_updated sector  \
type year month geography                                                
AOR  2001 1     USA-AK     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-AL     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-AR     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-AZ     201706  M  2017-08-24T11:46:12-04:00     99   
                USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   

                                      series_id   start          units  \
type year month geography                                                
AOR  2001 1     USA-AK     ELEC.GEN.AOR-AK-99.M  200101  megawatthours   
                USA-AL     ELEC.GEN.AOR-AL-99.M  200101  megawatthours   
                USA-AR     ELEC.GEN.AOR-AR-99.M  200101  megawatthours   
                USA-AZ     ELEC.GEN.AOR-AZ-99.M  200101  megawatthours   
                USA-CA     ELEC.GEN.AOR-CA-99.M  200101  megawatthours   

                           generation (MWh)  total fuel (mmbtu)  \
type year month geography                                         
AOR  2001 1     USA-AK                87.00                 NaN   
                USA-AL            401167.59                 NaN   
                USA-AR            136530.37                 NaN   
                USA-AZ               453.00                 NaN   
                USA-CA           1717398.41                 NaN   

                           elec fuel (mmbtu)  all fuel CO2 (kg)  \
type year month geography                                         
AOR  2001 1     USA-AK                   NaN                NaN   
                USA-AL                   NaN                NaN   
                USA-AR                   NaN                NaN   
                USA-AZ                   NaN                NaN   
                USA-CA                   NaN                NaN   

                           elec fuel CO2 (kg)  
type year month geography                      
AOR  2001 1     USA-AK                    NaN  
                USA-AL                    NaN  
                USA-AR                    NaN  
                USA-AZ                    NaN  
                USA-CA                    NaN

In [183]:
years = gen_fuel_df.index.get_level_values('year').astype(str)
months = gen_fuel_df.index.get_level_values('month').astype(str)
datetime = pd.to_datetime(years + '-' + months, format='%Y-%m')
quarters = datetime.quarter

gen_fuel_df['datetime'] = datetime
gen_fuel_df['quarter'] = quarters

No records with positive fuel use but no generation

In [186]:
gen_fuel_df['generation (MWh)'].fillna(value=0, inplace=True)

In [189]:
#drop
gen_fuel_df.loc[gen_fuel_df['sector'].isnull()]

end    f last_updated sector series_id start units  \
type year month geography                                                       
COW  2016 1     USA-ID     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          3     USA-ID     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2017 3     USA-AK     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-ME     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          4     USA-AK     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          5     USA-AK     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          6     USA-AK     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
NG   2010 4     USA-MT     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          8     USA-MT     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2015 2     USA-NE     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2017 3     USA-ND     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          4     USA-ND     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          5     USA-ME     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-ND     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          6     USA-ND     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
PC   2011 4     USA-NY     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          6     USA-NY     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2016 1     USA-MI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-OH     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-TX     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          2     USA-IA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          4     USA-IA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          7     USA-IA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          12    USA-IA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2017 3     USA-OH     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
PEL  2012 8     USA-OK     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2013 1     USA-RI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          8     USA-RI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          11    USA-RI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          12    USA-RI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2014 7     USA-RI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2015 3     USA-RI     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          6     USA-CO     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
     2016 1     USA-NH     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          2     USA-NH     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          3     USA-ME     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-WA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          4     USA-KS     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-ME     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-MS     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-SC     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-WA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          5     USA-ME     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-WA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          9     USA-ME     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          10    USA-WA     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
          11    USA-CT     NaN  NaN          NaN    NaN       NaN   NaN   NaN   
                USA-ME     NaN  NaN          NaN    NaN   

In [190]:
gen_fuel_df.loc['COW',:].head()

end  f               last_updated sector  \
year month geography                                                
2001 1     USA-AK     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-AL     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-AR     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-AZ     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   

                                 series_id   start          units  \
year month geography                                                
2001 1     USA-AK     ELEC.GEN.COW-AK-99.M  200101  megawatthours   
           USA-AL     ELEC.GEN.COW-AL-99.M  200101  megawatthours   
           USA-AR     ELEC.GEN.COW-AR-99.M  200101  megawatthours   
           USA-AZ     ELEC.GEN.COW-AZ-99.M  200101  megawatthours   
           USA-CA     ELEC.GEN.COW-CA-99.M  200101  megawatthours   

                      generation (MWh)  total fuel (mmbtu)  elec fuel (mmbtu)  \
year month geography                                                            
2001 1     USA-AK              46903.0           1120000.0           872000.0   
           USA-AL            6557913.0          67999000.0         66582000.0   
           USA-AR            2149808.0          23099000.0         22700000.0   
           USA-AZ            3418454.0          35873000.0         35483000.0   
           USA-CA             199857.0           3652000.0          2008000.0   

                      all fuel CO2 (kg)  elec fuel CO2 (kg)   datetime  \
year month geography                                                     
2001 1     USA-AK          1.066800e+08        8.305800e+07 2001-01-01   
           USA-AL          6.476905e+09        6.341935e+09 2001-01-01   
           USA-AR          2.200180e+09        2.162175e+09 2001-01-01   
           USA-AZ          3.416903e+09        3.379756e+09 2001-01-01   
           USA-CA          3.478530e+08        1.912620e+08 2001-01-01   

                      quarter  
year month geography           
2001 1     USA-AK           1  
           USA-AL           1  
           USA-AR           1  
           USA-AZ           1  
           USA-CA           1

In [191]:
#drop
gen_fuel_df.loc['OOG'].head()

end  f               last_updated sector  \
year month geography                                                
2001 1     USA-AL     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-CA     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-CT     201012  M  2016-12-19T17:19:30-05:00     99   
           USA-DE     201706  M  2017-08-24T11:46:12-04:00     99   
           USA-FL     201706  M  2017-08-24T11:46:12-04:00     99   

                                 series_id   start          units  \
year month geography                                                
2001 1     USA-AL     ELEC.GEN.OOG-AL-99.M  200101  megawatthours   
           USA-CA     ELEC.GEN.OOG-CA-99.M  200101  megawatthours   
           USA-CT     ELEC.GEN.OOG-CT-99.M  200101  megawatthours   
           USA-DE     ELEC.GEN.OOG-DE-99.M  200101  megawatthours   
           USA-FL     ELEC.GEN.OOG-FL-99.M  200101  megawatthours   

                      generation (MWh)  total fuel (mmbtu)  elec fuel (mmbtu)  \
year month geography                                                            
2001 1     USA-AL              25283.0                 NaN                NaN   
           USA-CA              97569.0                 NaN                NaN   
           USA-CT                137.0                 NaN                NaN   
           USA-DE              12552.0                 NaN                NaN   
           USA-FL                  9.0                 NaN                NaN   

                      all fuel CO2 (kg)  elec fuel CO2 (kg)   datetime  \
year month geography                                                     
2001 1     USA-AL                   NaN                 NaN 2001-01-01   
           USA-CA                   NaN                 NaN 2001-01-01   
           USA-CT                   NaN                 NaN 2001-01-01   
           USA-DE                   NaN                 NaN 2001-01-01   
           USA-FL                   NaN                 NaN 2001-01-01   

                      quarter  
year month geography           
2001 1     USA-AL           1  
           USA-CA           1  
           USA-CT           1  
           USA-DE           1  
           USA-FL           1

### Export data

In [192]:
path = join(cwd, '..', 'Data storage', 'EIA state-level gen fuel CO2.csv')
gen_fuel_df.to_csv(path, index=False)